In [298]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.model_selection import train_test_split

#Use database 
import sqlite3
conn = sqlite3.connect('example.db')

# Rosbank ML Competition

- Данные

Датасет, который содержит историю транзакций клиентов за 3 месяца льготного использования банковского продукта

- Задача

*Задача* бинарной классификации – прогноз оттока клиентов

Колонка cl_id содержит вутренний id клиента. Для каждого уникальнго cl_id следует предсказать продолжит ли клиент пользоваться продуктом (target_flag). Значение 0 соответствует отказу, а значение 1 соответствует продолжению использования

### Описание данных
- trx_category

Вид транзакции, POS – оплата через POS терминал, C2C_OUT – перевод на карту (исходящий платёж), C2C_IN – перевод на карту (входящий платёж), DEPOSIT – пополнение карты в банкомате, WD_ATM_PARTNER – снятие наличных в банкоматах партнерах,

In [341]:
df = pd.read_csv('./data/train.csv')

def convert_TRDATETIME(dt):
    new_dt=datetime.strptime(dt, '%d%b%y:%H:%M:%S')
    return int(new_dt.strftime('%Y%m%d%H%M%S'))

def convert_PERIOD(dt):
    new_dt=datetime.strptime(dt, '%d/%m/%Y')
    return int(new_dt.strftime('%Y%m%d'))
    
    
df['TRDATETIME'] = train.apply(lambda x: convert_TRDATETIME(x['TRDATETIME']), axis=1)
df['PERIOD'] = train.apply(lambda x: convert_PERIOD(x['PERIOD']), axis=1)

In [342]:
df.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,20170201.0,0,5200,NaN,810,2.017020e+13,5023.0,POS,0,0.0
1,20170201.0,0,6011,NaN,810,2.017021e+13,20000.0,DEPOSIT,0,0.0
2,20170201.0,0,5921,NaN,810,2.017022e+13,767.0,POS,0,0.0
3,20170201.0,0,5411,NaN,810,2.017023e+13,2031.0,POS,0,0.0
4,20170201.0,0,6012,NaN,810,2.017022e+13,36562.0,C2C_OUT,0,0.0


In [324]:
# Дополнительные данные с MCC-кодами и валютами
mcc = pd.read_csv('./data/mcc_codes_alfa.csv')
currencies = pd.read_csv('./data/currencies.csv')

In [345]:
mcc.head()

,mcc,mcc_name,mcc_group,big_cashback,no_cashback
0,742,Ветеринарные услуги,Контрактные услуги,0,0
1,763,Сельскохозяйственные кооперативы,Контрактные услуги,0,0
2,780,Ландшафтные и садоводческие магазины,Контрактные услуги,0,0
3,1520,Генеральные подрядчики – жилищное и торговое с...,Контрактные услуги,0,0
4,1711,"Генеральные подрядчики по вентиляции, теплосна...",Контрактные услуги,0,0


In [347]:
currencies.head()

,Entity,Currency,AlphabeticCode,NumericCode,MinorUnit,WithdrawalDate
0,AFGHANISTAN,Afghani,AFN,971.0,2,NaN
1,ÅLAND ISLANDS,Euro,EUR,978.0,2,NaN
2,ALBANIA,Lek,ALL,8.0,2,NaN
3,ALGERIA,Algerian Dinar,DZD,12.0,2,NaN
4,AMERICAN SAMOA,US Dollar,USD,840.0,2,NaN


1. Наборы данных вида Transactions (несколько транзакций на одного клиента) трансформировать в таблицу, где cl_id будут уникальными (соответственно 4000 строк в train и 1000 строк в test

2. Для каждого cl_id будет уникальное целевое событие target_flag, а также уникальный канал привлечения клиента channel_type (клиент привлекается лишь однажды и с самого начала его записи присваивается значение канала привлечения)

3. При агрегации (*pandas.DataFrame.groupby*) по cl_id (или по связке cl_id, channel_type, target_flag) необходимо создавать производные фичи, идеи для таких фичей могут быть следующими:

    - общая сумма транзакций по каждой из trx_category
    - общая сумма транзакции по основным вылютам (напр. выделить рубли, доллары и евро - предположительно, это будут самые крупные категории)
    - общая сумма транзакций по категориям MCC кодов (например, выбрать основные/популярные MCC коды). ВНИМАНИ! Некоторые MCC коды из train могут быть не представлены в test. Про MCC коды в целом: http://www.banki.ru/wikibank/mcc-kod/; Справочник MCC кодов: https://mcc-codes.ru/code; Про некоторые категории кэшбека Росбанка: https://mcc-codes.ru/card/rosbank-sverkh-plus;
    - возможные агрегации по времени суток и дням недели - траты в выходные (праздники) или будни, в ночное время или в рабочее и т.д.

In [348]:
def client_split_train_test(df):
    cl_ids = df[['cl_id', 'channel_type', 'target_flag']].drop_duplicates()
    cl_train, cl_test = train_test_split(cl_ids, test_size=0.2, random_state=42 )
    train = cl_train[['cl_id']].merge(df, how='inner', on='cl_id')
    test = cl_test[['cl_id']].merge(df, how='inner', on='cl_id')
    return train, test

train, test = client_split_train_test(df)
train.head()

,cl_id,PERIOD,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,8729,NaN,7311,type1,810,NaN,5000.00,POS,1,8791.85
1,8729,NaN,7230,type1,810,NaN,800.00,POS,1,8791.85
2,8729,NaN,5411,type1,810,NaN,1837.14,POS,1,8791.85
3,8729,NaN,5411,type1,810,NaN,2610.71,POS,1,8791.85
4,8729,NaN,5812,type1,810,NaN,1583.40,POS,1,8791.85


In [349]:
#Use database
import sqlite3
conn = sqlite3.connect('rosbank.db')
mcc.to_sql('mcc', conn, if_exists='replace')
currencies.to_sql('currencies', conn, if_exists='replace')
train.to_sql('train', conn, if_exists='replace')
test.to_sql('test', conn, if_exists='replace')

### Гипотезы
На лояльность клиента влияют следующие факторы:

1. Сумма совершенных покупок по программам лояльности
2. Сумма транзакций по trx категориям
3. Сумма покупок по группам mcc категорий
4. Сумма покупок совершенных без кэшбэка
5. Медианное количество POS-транзакций в месяц
6. Медианное количество денег, переведенное на карту клиента
7. Количество маленьких покупок совершенных по карте в месяц (сумма до 1000 рублей)

In [329]:
# Лояльность по количеству покупок
df_median=pd.read_sql("""Select cl_id,
                                channel_type, 
                                target_flag, 
                                SUM(big_cashback) as big_cashback_cnt, 
                                SUM(no_cashback) as no_cashback_cnt, 
                                SUM(case when trx_category == 'POS' then 1 else 0 end) as pos_cnt ,
                                SUM(case when trx_category != 'POS' then 1 else 0 end) as no_pos_cnt ,
                                SUM(case when trx_category in ('C2C_IN', 'DEPOSIT') then 1 else 0 end) as c2c_in_amnt,
                                SUM(case when amount<=1000 then 1 else 0 end) as small_transactions_cnt
                    FROM train LEFT JOIN mcc
                        ON mcc.mcc = train.MCC
                GROUP BY cl_id, channel_type, target_flag""", conn)
df_median.head()

,cl_id,channel_type,target_flag,big_cashback_cnt,no_cashback_cnt,pos_cnt,no_pos_cnt,c2c_in_amnt,small_transactions_cnt
0,0,None,0,1,2,3,2,1,1
1,1,None,0,76,5,101,3,2,89
2,5,None,1,29,37,111,31,4,48
3,9,None,0,0,10,29,10,4,26
4,10,None,0,100,91,374,89,25,370


In [330]:
# Категории транзакций
table = pd.read_sql("""SELECT cl_id, 
                               CASE 
                                   WHEN mcc_group NOT IN 
                                               ('Коммунальные и кабельные услуги',
                                                'Транспорт',
                                                'Автомобили и транспортные средства',
                                                'Развлечения',
                                                'Личные услуги',
                                                'Розничные магазины',
                                                'Поставщик услуг',
                                                'Магазины одежды',
                                                'Различные магазины') 
                                    THEN 'OTHER'
                                    else mcc_group END mcc_group, 
                                SUM(amount) amnt, COUNT(1) cnt 
                        FROM mcc INNER JOIN train
                            ON mcc.mcc = train.MCC
                            GROUP BY cl_id, mcc_group
                            ORDER BY cl_id, cnt DESC
                    """, conn)

table = table.pivot_table(values='cnt', index=['cl_id'], columns=['mcc_group']).fillna(value=0).reset_index()

In [331]:
# Используемые валюты
df_currency = pd.read_sql("""SELECT cl_id,
                      CASE WHEN currencies.Currency 
                            NOT IN ('Russian Ruble', 'Euro', 'US Dollar') 
                           THEN 'OTHER'
                           ELSE currencies.Currency 
                       END currency,
                       1 as cnt FROM currencies 
                                        INNER JOIN train ON train.currency = currencies.NumericCode
                            WHERE trx_category == 'POS'
                        GROUP BY 1, 2
                        ORDER BY 2 DESC
            """, conn)\
            .pivot_table(values='cnt', index=['cl_id'], columns=['currency']).fillna(value=0).reset_index()

In [332]:
train.head()

,cl_id,PERIOD,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,8729,01/02/2017,7311,type1,810,03Feb17:00:00:00,5000.00,POS,1,8791.85
1,8729,01/02/2017,7230,type1,810,11Feb17:00:00:00,800.00,POS,1,8791.85
2,8729,01/02/2017,5411,type1,810,21Feb17:00:00:00,1837.14,POS,1,8791.85
3,8729,01/02/2017,5411,type1,810,27Feb17:00:00:00,2610.71,POS,1,8791.85
4,8729,01/02/2017,5812,type1,810,22Feb17:00:00:00,1583.40,POS,1,8791.85


In [336]:
# Время транзакций



0         20170203000000
1         20170211000000
2         20170221000000
3         20170227000000
4         20170222000000
5         20170307000000
6         20170326104104
7         20170404000000
8         20170401093535
9         20170317000000
10        20170308000000
11        20170304000000
12        20170228230450
13        20170216000000
14        20170212000000
15        20170208000000
16        20170126000000
17        20170330000000
18        20170319130057
19        20170314000000
20        20170308000000
21        20170307000000
22        20170305000000
23        20170226172827
24        20170228230733
25        20170228000000
26        20170302000000
27        20170220000000
28        20170214000000
29        20170210000000
               ...      
392788    20161216000000
392789    20161226000000
392790    20170214000000
392791    20170203000000
392792    20170121000000
392793    20161127000000
392794    20170109000000
392795    20170217000000
392796    20170210000000


In [290]:
#Все фичи
df_full = df_median\
            .merge(table, how='inner', on='cl_id')\
            .merge(df_currency, how='inner', on='cl_id')


In [291]:
corr = df_full.corr()
corr

,cl_id,target_flag,big_cashback_cnt,no_cashback_cnt,pos_cnt,no_pos_cnt,c2c_in_amnt,small_transactions_cnt,OTHER_x,Автомобили и транспортные средства,...,Магазины одежды,Поставщик услуг,Развлечения,Различные магазины,Розничные магазины,Транспорт,Euro,OTHER_y,Russian Ruble,US Dollar
cl_id,1.000000,-0.370619,-0.143726,-0.109077,-0.108941,-0.118273,0.184088,-0.093148,0.031178,-0.012885,...,-0.102501,-0.117184,-0.042088,-0.152177,-0.031655,-0.099430,-0.136829,-0.133963,-0.006528,-0.004271
target_flag,-0.370619,1.000000,0.248601,0.184104,0.245932,0.169439,-0.078115,0.200082,0.066024,0.117192,...,0.177039,0.169233,0.117312,0.261118,0.142100,0.115962,0.121372,0.134278,0.071501,0.012866
big_cashback_cnt,-0.143726,0.248601,1.000000,0.294942,0.874717,0.267183,0.169855,0.846237,0.246505,0.338083,...,0.460305,0.261333,0.351835,0.910214,0.614430,0.308532,0.161468,0.123101,0.075499,0.062100
no_cashback_cnt,-0.109077,0.184104,0.294942,1.000000,0.380482,0.936657,0.509370,0.382015,0.222301,0.214835,...,0.165603,0.946278,0.207678,0.306984,0.260449,0.165854,0.034831,0.041921,0.089664,0.026217
pos_cnt,-0.108941,0.245932,0.874717,0.380482,1.000000,0.318316,0.269890,0.962264,0.330099,0.485584,...,0.480610,0.313446,0.363308,0.849782,0.836929,0.388339,0.152145,0.120693,0.090568,0.090238
no_pos_cnt,-0.118273,0.169439,0.267183,0.936657,0.318316,1.000000,0.535659,0.334731,0.094702,0.192282,...,0.164286,0.985653,0.132431,0.267751,0.246443,0.142667,0.012635,0.029659,0.088549,0.002720
c2c_in_amnt,0.184088,-0.078115,0.169855,0.509370,0.269890,0.535659,1.000000,0.274706,0.138831,0.231684,...,0.107111,0.535534,0.086196,0.149331,0.284497,0.051721,-0.050615,-0.053389,0.066249,-0.005803
small_transactions_cnt,-0.093148,0.200082,0.846237,0.382015,0.962264,0.334731,0.274706,1.000000,0.291086,0.427610,...,0.405641,0.325748,0.332659,0.820261,0.826143,0.402794,0.132381,0.101450,0.068990,0.089051
OTHER_x,0.031178,0.066024,0.246505,0.222301,0.330099,0.094702,0.138831,0.291086,1.000000,0.192134,...,0.141685,0.087063,0.086971,0.224784,0.203046,0.102826,0.071508,0.079156,0.007597,0.101292
Автомобили и транспортные средства,-0.012885,0.117192,0.338083,0.214835,0.485584,0.192282,0.231684,0.427610,0.192134,1.000000,...,0.163076,0.190885,0.117819,0.311469,0.392995,0.026666,0.047864,0.066964,0.054848,0.007297


In [292]:
corr.target_flag.sort_values()

cl_id                                -0.370619
c2c_in_amnt                          -0.078115
US Dollar                             0.012866
OTHER_x                               0.066024
Russian Ruble                         0.071501
Коммунальные и кабельные услуги       0.102412
Транспорт                             0.115962
Автомобили и транспортные средства    0.117192
Развлечения                           0.117312
Euro                                  0.121372
OTHER_y                               0.134278
Личные услуги                         0.139587
Розничные магазины                    0.142100
Поставщик услуг                       0.169233
no_pos_cnt                            0.169439
Магазины одежды                       0.177039
no_cashback_cnt                       0.184104
small_transactions_cnt                0.200082
pos_cnt                               0.245932
big_cashback_cnt                      0.248601
Различные магазины                    0.261118
target_flag  